# DATASCI W261: Machine Learning at Scale 

**Name: Carlos Eduardo Rodriguez Castillo**

**email: cerodriguez@berkeley.edu**

**Week 1**

**Section 2**


### This notebook provides a poor man Hadoop through command-line and python. Please note that I kept logging commented logging code to show my work.

#### Map

In [1]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
#import logging

#logging_directory_path = '/home/crodriguez1/test_log/'
#logging_file = logging_daily_directory_path + "/" + LOGGING_FILE_NAME
#logging_file = logging_directory_path + "/" + 'test_map.log'
#log_format = '%(levelname)s\n%(asctime)s.%(msecs)-3d filename:%(filename)-20s line:%(lineno)-5d \n%(message)s\n\n'
#log_date_format = '%H:%M:%S'
#logging.basicConfig(filename = logging_file,
#    stream = sys.stderr,
#    level = logging.DEBUG,
#    format = log_format,
#    datefmt = log_date_format)
#suds_logger = logging.getLogger("suds")
#suds_logger.propagate = False

count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[2]
findword = sys.argv[1]
with open (filename, "r") as myfile:
    count = 0
#Please insert your code
    for line in myfile:
        #logging.debug('Line: %s'% line)
        words = line.split()
        for word in words:
            #logging.debug('Word: %s' % word)
            word = word.rstrip(',')
            word = word.rstrip(';')
            word = word.rstrip(':')
            word = word.rstrip('.')
            if word == findword:
                #print count
                #logging.debug('Word: [%s] is equal to [%s]! Increment!' % (word, findword))
                count = count + 1
    print count

Overwriting mapper.py


In [2]:
!chmod a+x mapper.py

#### Reduce

In [3]:
%%writefile reducer.py
#!/usr/bin/python
import sys
#import logging

#logging_directory_path = '/home/crodriguez1/test_log/'
#logging_file = logging_daily_directory_path + "/" + LOGGING_FILE_NAME
#logging_file = logging_directory_path + "/" + 'test_reduce.log'
#log_format = '%(levelname)s\n%(asctime)s.%(msecs)-3d filename:%(filename)-20s line:%(lineno)-5d \n%(message)s\n\n'
#log_date_format = '%H:%M:%S'
#logging.basicConfig(filename = logging_file,
#    stream = sys.stderr,
#    level = logging.DEBUG,
#    format = log_format,
#    datefmt = log_date_format)
#suds_logger = logging.getLogger("suds")
#suds_logger.propagate = False

sum = 0
for line in sys.stdin:
#Please insert your code
    #logging.debug('Sum line: %s'% line)
    sum = sum + int(line)
    #logging.debug('Sum: %s ' % sum)
print sum

Overwriting reducer.py


In [4]:
!chmod a+x reducer.py

## Write script to file

In [5]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Overwriting pGrepCount.sh


#### Run the file

In [6]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [7]:
!./pGrepCount.sh LICENSE.txt COPYRIGHT 4k

found [11] [COPYRIGHT] in the file [LICENSE.txt]
